In [1]:
import sys
from pathlib import Path

reporoot = Path(".").absolute().parent


In [2]:
sys.path.append(str(reporoot))


In [3]:
from c2st.check import c2st_


In [4]:
help(c2st)

NameError: name 'c2st' is not defined

In [12]:
from __future__ import annotations
import numpy as np
from functools import partial
from numpy.random import default_rng
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn import __version__ as sklversion
import time
FIXEDSEED = 1309

In [6]:
def nn_c2st_(
    X: np.ndarray,
    Y: np.ndarray,
    seed: int = FIXEDSEED,
    n_folds: int = 5,
    scoring: str = "accuracy",
    z_score: bool = True,
    noise_scale: Optional[float] = None,
    verbosity: int = 0,
) -> np.ndarray:

    ndim = X.shape[1]
    clf_class = MLPClassifier
    clf_kwargs = {
        "activation": "relu",
        "hidden_layer_sizes": (10 * ndim, 10 * ndim),
        "max_iter": 1000,
        "solver": "adam",
    }

    return c2st_(
        X,
        Y,
        seed,
        n_folds,
        scoring,
        z_score,
        noise_scale,
        verbosity,
        clf_class,
        clf_kwargs,
    )

In [7]:
NDIM = 10
max_nsamples = 8096
sample_sizes = [ 2**it for it in range(7,12)]
sample_sizes.append(max_nsamples)
RNG = default_rng(FIXEDSEED)
print(sample_sizes)

[128, 256, 512, 1024, 2048, 8096]


In [8]:
center_normal = partial(RNG.multivariate_normal, mean=np.zeros(NDIM), cov=np.eye(NDIM))
distributions = { 0. : center_normal}

for alpha in np.linspace(0,2,9):
    distributions[alpha] = partial(RNG.multivariate_normal, mean=np.zeros(NDIM) + alpha, cov=np.eye(NDIM))



In [9]:
center_samples = center_normal(size=max_nsamples)
samples = {}
for k,v in distributions.items():
    samples[k] = v(size=max_nsamples)

assert samples[0.].shape == (8096, 10)

In [10]:
rf_results = {}
rf_timings = {}
total = len(samples.values())*len(sample_sizes)
cnt = 0


for k,v in samples.items():
    for size in sample_sizes:

        try:
            start = time.time()
            scores = c2st_(X = center_samples[:size,...],
                          Y = v[:size,...],
                          n_folds=10)
            end = time.time()
        except Exception as ex:
            print(ex)
            continue

        if not k in rf_results.keys():
            rf_results[k] = []
            rf_timings[k] = []

        mean = np.mean(scores)
        std = np.std(scores)
        rf_results[k].append((mean,std))
        rf_timings[k].append(end-start)
        cnt += 1

        print(f"{cnt}/{total}: {k}[{size},...] = {mean,std} ({end-start} seconds)")

1/54: 0.0[128,...] = (0.42153846153846153, 0.12246385772832794) (1.0489280223846436 seconds)


2/54: 0.0[256,...] = (0.47835595776772244, 0.07806781384146226) (1.3461580276489258 seconds)


3/54: 0.0[512,...] = (0.5009232819341329, 0.048259547167832775) (2.162215232849121 seconds)


4/54: 0.0[1024,...] = (0.48974414155906265, 0.019876207761461956) (4.205496311187744 seconds)


5/54: 0.0[2048,...] = (0.5007263402707377, 0.01581535273212949) (9.193150758743286 seconds)


...

50/54: 2.0[256,...] = (0.998076923076923, 0.0057692307692307826) (1.078972339630127 seconds)


51/54: 2.0[512,...] = (0.9980487340567296, 0.00390258992495368) (1.5362026691436768 seconds)


52/54: 2.0[1024,...] = (0.9985365853658535, 0.002235402778027241) (2.6288297176361084 seconds)


53/54: 2.0[2048,...] = (0.998533603673445, 0.0019556928463501473) (5.543246269226074 seconds)


54/54: 2.0[8096,...] = (0.9986412508864639, 0.0006052481616375465) (28.65970253944397 seconds)


In [13]:
header = "ndims,mode,dist_sigma,nsamples,mean_c2st_score,std_c2st_score,time_sec,nfolds,sklearn_version"
with open("rf_results.csv","w") as ocsv:
    ocsv.write(header+"\n")
    for k in samples.keys():
        for idx in range(len(sample_sizes)):
            mn,std = rf_results[k][idx]
            row = f"{NDIM},rf,{k},{sample_sizes[idx]},{mn},{std},{rf_timings[k][idx]},10,{sklversion}\n"
            print(row)
            ocsv.write(row)

10,rf,0.0,128,0.42153846153846153,0.12246385772832794,1.0489280223846436,10,1.0.2

10,rf,0.0,256,0.47835595776772244,0.07806781384146226,1.3461580276489258,10,1.0.2

10,rf,0.0,512,0.5009232819341329,0.048259547167832775,2.162215232849121,10,1.0.2

10,rf,0.0,1024,0.48974414155906265,0.019876207761461956,4.205496311187744,10,1.0.2

10,rf,0.0,2048,0.5007263402707377,0.01581535273212949,9.193150758743286,10,1.0.2

10,rf,0.0,8096,0.5003095189074188,0.007796416628318716,50.5123770236969,10,1.0.2

10,rf,0.25,128,0.6201538461538462,0.0922985892947385,2.1747143268585205,10,1.0.2

10,rf,0.25,256,0.5820889894419305,0.053563623821549874,1.361140251159668,10,1.0.2

10,rf,0.25,512,0.5878926327812678,0.044028780012037166,2.0011892318725586,10,1.0.2

10,rf,0.25,1024,0.6147321855571497,0.03028625429776028,3.710228204727173,10,1.0.2

10,rf,0.25,2048,0.6310996481602957,0.02729351243745489,7.187965631484985,10,1.0.2

10,rf,0.25,8096,0.6420456919756898,0.009978095785152703,31.963542938232422,10,1.0.2

10,r

In [14]:
!head -n10 rf_results.csv

ndims,mode,dist_sigma,nsamples,mean_c2st_score,std_c2st_score,time_sec,nfolds,sklearn_version
10,rf,0.0,128,0.42153846153846153,0.12246385772832794,1.0489280223846436,10,1.0.2
10,rf,0.0,256,0.47835595776772244,0.07806781384146226,1.3461580276489258,10,1.0.2
10,rf,0.0,512,0.5009232819341329,0.048259547167832775,2.162215232849121,10,1.0.2
10,rf,0.0,1024,0.48974414155906265,0.019876207761461956,4.205496311187744,10,1.0.2
10,rf,0.0,2048,0.5007263402707377,0.01581535273212949,9.193150758743286,10,1.0.2
10,rf,0.0,8096,0.5003095189074188,0.007796416628318716,50.5123770236969,10,1.0.2
10,rf,0.25,128,0.6201538461538462,0.0922985892947385,2.1747143268585205,10,1.0.2
10,rf,0.25,256,0.5820889894419305,0.053563623821549874,1.361140251159668,10,1.0.2
10,rf,0.25,512,0.5878926327812678,0.044028780012037166,2.0011892318725586,10,1.0.2


In [15]:
nn_results = {}
nn_timings = {}
cnt = 0

for k,v in samples.items():
    for size in sample_sizes:

        try:
            start = time.time()
            scores = nn_c2st_(center_samples[:size,...],
                             v[:size,...])
            end = time.time()
        except Exception as ex:
            print(ex)
            continue
        
        if not k in nn_results.keys():
            nn_results[k] = []
            nn_timings[k] = []

        mean = np.mean(scores)
        std = np.std(scores)
        nn_results[k].append((mean,std))
        nn_timings[k].append(end-start)
        cnt += 1

        print(f"{cnt}/{total}: {k}[{size},...] = {mean,std} ({end-start} seconds)")

1/54: 0.0[128,...] = (0.47639517345399707, 0.06944370621739751) (3.9503777027130127 seconds)


2/54: 0.0[256,...] = (0.47270131353512274, 0.014611290821653794) (14.877474546432495 seconds)


3/54: 0.0[512,...] = (0.46679100908656146, 0.02653006985499437) (27.8264639377594 seconds)


4/54: 0.0[1024,...] = (0.4936513805235852, 0.01827138663151398) (62.5630042552948 seconds)


5/54: 0.0[2048,...] = (0.49633878316805147, 0.013397183951439668) (132.11983609199524 seconds)


...

50/54: 2.0[256,...] = (1.0, 0.0) (1.5427064895629883 seconds)


51/54: 2.0[512,...] = (0.9980439980870397, 0.002395615245328274) (4.2837560176849365 seconds)


52/54: 2.0[1024,...] = (0.9985365853658535, 0.0029268292682926855) (6.391915321350098 seconds)


53/54: 2.0[2048,...] = (0.9987789987789988, 0.0010920966923075857) (13.088403701782227 seconds)


54/54: 2.0[8096,...] = (0.9983324755179359, 0.000315090857989144) (44.79178190231323 seconds)


In [16]:
with open("nn_results.csv","w") as ocsv:
    ocsv.write(header+"\n")
    for k in samples.keys():
        for idx in range(len(sample_sizes)):
            mn,std = rf_results[k][idx]
            row = f"{NDIM},rf,{k},{sample_sizes[idx]},{mn},{std},{nn_timings[k][idx]},10,{sklversion}\n"
            print(row)
            ocsv.write(row)

10,rf,0.0,128,0.42153846153846153,0.12246385772832794,3.9503777027130127,10,1.0.2

10,rf,0.0,256,0.47835595776772244,0.07806781384146226,14.877474546432495,10,1.0.2

10,rf,0.0,512,0.5009232819341329,0.048259547167832775,27.8264639377594,10,1.0.2

10,rf,0.0,1024,0.48974414155906265,0.019876207761461956,62.5630042552948,10,1.0.2

10,rf,0.0,2048,0.5007263402707377,0.01581535273212949,132.11983609199524,10,1.0.2

10,rf,0.0,8096,0.5003095189074188,0.007796416628318716,463.0335690975189,10,1.0.2

10,rf,0.25,128,0.6201538461538462,0.0922985892947385,9.77917218208313,10,1.0.2

10,rf,0.25,256,0.5820889894419305,0.053563623821549874,27.03339195251465,10,1.0.2

10,rf,0.25,512,0.5878926327812678,0.044028780012037166,36.92687106132507,10,1.0.2

10,rf,0.25,1024,0.6147321855571497,0.03028625429776028,69.17806816101074,10,1.0.2

10,rf,0.25,2048,0.6310996481602957,0.02729351243745489,152.62547278404236,10,1.0.2

10,rf,0.25,8096,0.6420456919756898,0.009978095785152703,582.9192526340485,10,1.0.2

10,rf,0

In [17]:
!head -n10 nn_results.csv

ndims,mode,dist_sigma,nsamples,mean_c2st_score,std_c2st_score,time_sec,nfolds,sklearn_version
10,rf,0.0,128,0.42153846153846153,0.12246385772832794,3.9503777027130127,10,1.0.2
10,rf,0.0,256,0.47835595776772244,0.07806781384146226,14.877474546432495,10,1.0.2
10,rf,0.0,512,0.5009232819341329,0.048259547167832775,27.8264639377594,10,1.0.2
10,rf,0.0,1024,0.48974414155906265,0.019876207761461956,62.5630042552948,10,1.0.2
10,rf,0.0,2048,0.5007263402707377,0.01581535273212949,132.11983609199524,10,1.0.2
10,rf,0.0,8096,0.5003095189074188,0.007796416628318716,463.0335690975189,10,1.0.2
10,rf,0.25,128,0.6201538461538462,0.0922985892947385,9.77917218208313,10,1.0.2
10,rf,0.25,256,0.5820889894419305,0.053563623821549874,27.03339195251465,10,1.0.2
10,rf,0.25,512,0.5878926327812678,0.044028780012037166,36.92687106132507,10,1.0.2
